# Initialising LUSID

In [1]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd

import helper_functions as fun 

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')
print ('LUSID API Version: ', client.metadata.get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version:  0.5.3313.0


In [ ]:
# CAUTION: this block is only for experimental domains and should ony be used to de-clutter your domain

# WARNING: this deletes ALL existing portfolios FROM ALL SCOPES
# fun.delete_all_current_portfolios(client)

# WARNING: this deletes ALL existing instruments FROM ALL SCOPES
# fun.delete_all_current_instruments(client)

![Scopes](img/paper-lusid.gif)

## 1) Creating a logically separated environment for each analyst

In [2]:
# Fetch our scopes
analyst_scope_code = fun.create_analyst_scope()


Analyst Scope Code: analyst-paper-37c5-b819-20df-54


*Our LUSID environment now looks like the below. We will update this with each step.*

![Scopes](img/paper-scopes.gif)

## 2) Creating our tradeable instrument universe for each analyst

In [3]:
# Import our instrument universe from a CSV file using Pandas
instrument_universe = pd.read_csv('data/paper-instruments.csv')
# Look at the first 10 instruments
instrument_universe.head(n=10)

,instrument_name,currency,figi,ticker,isin,sedol,client_internal
0,AVEVA GRP,GBP,BBG000C21Y87,AVV,GB00BBG9VN75,BBG9VN7,imd_56344535
1,BAE SYS.,GBP,BBG000BD5TW4,BA.,GB0002634946,263494,imd_56344983
2,BATM ADVANCED,GBP,BBG000BFJD77,BVC,IL0010849045,911146,imd_56344098
3,BTG,GBP,BBG000BZZK79,BTG,GB0001001592,100159,imd_56344345
4,COBHAM,GBP,BBG000BS6810,COB,GB00B07KD360,B07KD36,imd_56344363
5,COMPUTACENTER,GBP,BBG000BN7CL9,CCC,GB00BV9FP302,BV9FP30,imd_56344296
6,CONSORT MED.,GBP,BBG000BD8278,CSRT,GB0000946276,94627,imd_56344187
7,DIALIGHT,GBP,BBG000BLQNG4,DIA,GB0033057794,3305779,imd_56344572
8,GENUS,GBP,BBG000CTMKX7,GNS,GB0002074580,207458,imd_56344789
9,GRESHAM TECH,GBP,BBG000BS2592,GHT,GB0008808825,880882,imd_56344132


In [4]:
batch_upsert_request = fun.batch_upsert(instrument_universe, 
                 client)   

Instruments Successfully Upserted: 


In [5]:
# Call LUSID to upsert our batch
instrument_response = client.instruments.upsert_instruments(
    requests=batch_upsert_request)

# Pretty print the response from LUSID
prettyprint.instrument_response(instrument_response, identifier='Figi')

Instruments Successfully Upserted: 


,Instrument,Figi ID,LUSID Instrument ID
0,SDL,BBG000C4HGK1,LUID_NBBPEAXL
1,SPIRENT,BBG000C4LV50,LUID_OZ9NME0G
2,CONSORT MED.,BBG000BD8278,LUID_TYAPFP0Q
3,XP POWER,BBG000BZMBQ2,LUID_88ZHDL5K
4,GRESHAM TECH,BBG000BS2592,LUID_REHCWF6R
5,AVEVA GRP,BBG000C21Y87,LUID_XA08PQYW
6,VECTURA,BBG000PRWXX8,LUID_YZJASMKH
7,SPECTRIS,BBG000CNV8Z5,LUID_M8EP0KPM
8,SAGE GRP.,BBG000BN0PP3,LUID_X6QSC75W
9,RENISHAW,BBG000BDZTQ8,LUID_XQBD5OMX


Our LUSID environment now has an instrument master containing all of our instruments that we will be trading.

![Scopes](img/paper-instrumentmaster.gif)

## 3) Creating a transactions portfolio for each analyst to hold their tradeable instrument universe and creating a reference portfolo for each analyst which contains the index of their tradeable instrument universe and can be used as a benchmark for performance

In [6]:
# Define unique codes for each of our portfolios
transaction_portfolio_code = 'LSE_FTSE_techMarkFocus_transactions_DEMO'
reference_portfolio_code = 'LSE_FTSE_techMarkFocus_reference_DEMO'
# Pretty print our codes
prettyprint.heading('Transaction Portfolio Code', transaction_portfolio_code)
prettyprint.heading('Reference Portfolio Code', reference_portfolio_code)

# The date our portfolios were first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=3))

description = 'Paper transaction portfolio_DEMO'
fun.request_transaction_portfolio_creation(transaction_portfolio_code, 
                                           portfolio_creation_date, 
                                           analyst_scope_code, 
                                           description,
                                           client)

Transaction Portfolio Code: LSE_FTSE_techMarkFocus_transactions_DEMO
Reference Portfolio Code: LSE_FTSE_techMarkFocus_reference_DEMO
Portfolio Created
Scope: analyst-paper-37c5-b819-20df-54
Code: LSE_FTSE_techMarkFocus_transactions_DEMO
Portfolio Effective From: 2019-09-27 13:57:15.231965+00:00
Portfolio Created On: 2019-09-30 13:57:15.478709+00:00



![Scopes](img/paper-transactionsportfolio.gif)

In [7]:
# The date our portfolios were first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=3))

fun.request_reference_portfolio_creation(reference_portfolio_code, 
                                         portfolio_creation_date, 
                                         analyst_scope_code, 
                                         client)


Portfolio Created
Scope: analyst-paper-37c5-b819-20df-54
Code: LSE_FTSE_techMarkFocus_reference_DEMO
Portfolio Effective From: 2019-09-27 13:57:17.996804+00:00
Portfolio Created On: 2019-09-30 13:57:18.368997+00:00



![Scopes](img/paper-referenceportfolio.gif)

## 4) Populating our analyst's portfolio with a starting cash balance

In [8]:
# Set the date from which the cash balance will apply to be just after portfolio creation
holdings_effective_date = (datetime.now(pytz.UTC) - timedelta(days=2.9))
# Define our initial cash balance
initial_cash_balance = 30000000

# # Create a holding adjustment to set our initial cash balance
fun.populate_with_cash(holdings_effective_date, 
                       initial_cash_balance, 
                       analyst_scope_code, 
                       transaction_portfolio_code, 
                       client)


Holdings Successfully Set for Portfolio
Scope: analyst-paper-37c5-b819-20df-54
Code: LSE_FTSE_techMarkFocus_transactions_DEMO
Holdings Effective From: 2019-09-27 16:21:21.437131+00:00
Holdings Created On: 2019-09-30 13:57:21.764164+00:00



Our transactions portfolio is now populated with an initial cash balance and is no longer empty.

![Scopes](img/paper-transactionsportfoliofilled.gif)

## 5) Populating our reference portfolio with the correct weights so that it tracks the index as closely as possible

In [9]:
# Import the market capitalisation of each of our instruments in the index
instrument_market_cap = pd.read_csv('data/paper-weights.csv')
# Take a look at the first 10 market capitalisations
instrument_market_cap.head(n=10)

,instrument_name,currency,figi,ticker,isin,sedol,marketcap
0,AVEVA GRP,GBP,BBG000C21Y87,AVV,GB00BBG9VN75,BBG9VN7,4502.68
1,BAE SYS.,GBP,BBG000BD5TW4,BA.,GB0002634946,0263494,16349.66
2,BATM ADVANCED,GBP,BBG000BFJD77,BVC,IL0010849045,911146,185.25
3,BTG,GBP,BBG000BZZK79,BTG,GB0001001592,100159,3202.20
4,COBHAM,GBP,BBG000BS6810,COB,GB00B07KD360,B07KD36,2579.90
5,COMPUTACENTER,GBP,BBG000BN7CL9,CCC,GB00BV9FP302,BV9FP30,1161.96
6,CONSORT MED.,GBP,BBG000BD8278,CSRT,GB0000946276,0094627,422.56
7,DIALIGHT,GBP,BBG000BLQNG4,DIA,GB0033057794,3305779,133.39
8,GENUS,GBP,BBG000CTMKX7,GNS,GB0002074580,0207458,1455.72
9,GRESHAM TECH,GBP,BBG000BS2592,GHT,GB0008808825,0880882,64.34


In [10]:
# upsert contituents to LUSID
fun.upsert_constituents(instrument_market_cap, 
                        holdings_effective_date, 
                        analyst_scope_code, 
                        reference_portfolio_code, 
                        client)

Constituents Upserted


![Scopes](img/paper-referenceportfoliofilled.gif)

## 6) Allow our analysts to trade across their tradeable instrument universe and add their transactions to their transaction portfolio

In [11]:
# Import our analyst's trades from a CSV file
analyst_transactions = import_data.fetch_client_transactions(
    'paper-transactions.csv',
    days_back=2)
analyst_transactions.head(n=20)

# Create a request to define our strategy property
domain = 'Transaction'
scope = analyst_scope_code
code='strategy'
display_name='strategy'

strategy_property_key = fun.request_define_property(domain, 
                                                    scope, 
                                                    code, 
                                                    display_name, 
                                                    client)


Strategy Property Key: : Transaction/analyst-paper-37c5-b819-20df-54/strategy


In [12]:
# upsert all analyist transactions to LUSID
fun.upsert_trades(analyst_transactions, 
                  strategy_property_key, 
                  analyst_scope_code, 
                  transaction_portfolio_code, 
                  client)

Transactions Successfully Upserted into Portfolio
Scope: analyst-paper-37c5-b819-20df-54
Code: LSE_FTSE_techMarkFocus_transactions_DEMO
Transactions Effective From: 2019-09-29 16:21:46.074797+00:00
Transactions Created On: 2019-09-30 13:57:33.310661+00:00



Our transaction portfolio is now populated with our analyst's paper trades.

## 7) Create an analytics store to hold the latest prices of our tradeable instrument universe

In [13]:
# Import our instrument prices from a CSV file
instrument_prices = pd.read_csv('data/paper-analytics.csv')
# Pretty print our pricing
instrument_prices.head(n=10)

,instrument_name,currency,figi,price_original,price_current,ticker,isin,sedol
0,AVEVA GRP,GBP,BBG000C21Y87,27.84,30.32,AVV,GB00BBG9VN75,BBG9VN7
1,BAE SYS.,GBP,BBG000BD5TW4,5.15,4.78,BA.,GB0002634946,263494
2,BATM ADVANCED,GBP,BBG000BFJD77,0.46,0.42,BVC,IL0010849045,911146
3,BTG,GBP,BBG000BZZK79,8.28,8.97,BTG,GB0001001592,100159
4,COBHAM,GBP,BBG000BS6810,1.07,1.02,COB,GB00B07KD360,B07KD36
5,COMPUTACENTER,GBP,BBG000BN7CL9,10.68,11.46,CCC,GB00BV9FP302,BV9FP30
6,CONSORT MED.,GBP,BBG000BD8278,8.70,9.31,CSRT,GB0000946276,94627
7,DIALIGHT,GBP,BBG000BLQNG4,3.92,3.55,DIA,GB0033057794,3305779
8,GENUS,GBP,BBG000CTMKX7,22.46,20.60,GNS,GB0002074580,207458
9,GRESHAM TECH,GBP,BBG000BS2592,0.94,1.00,GHT,GB0008808825,880882


In [14]:
# Set our analytics effective dates
analytics_effective_date = datetime.now(pytz.UTC) - timedelta(days=3)
today = datetime.now(pytz.UTC)
analytics_store_dates = [analytics_effective_date, today]

fun.create_instrument_analytics(analytics_effective_date, 
                                today, 
                                analytics_store_dates, 
                                instrument_prices, 
                                analyst_scope_code, 
                                client)
    

Analytics Set


![Scopes](img/paper-analyticsstore.gif)

## 8) Value each portfolio using our analytics store

In [15]:
# create aggregation request
aggregation_request = fun.create_aggregation_request(analyst_scope_code, today, transaction_portfolio_code, client)

# Call LUSID to aggregate across all of our portfolios
aggregated_portfolio = client.aggregation.get_aggregation_by_portfolio(
    scope=analyst_scope_code,
    code=transaction_portfolio_code,
    request=aggregation_request)

prettyprint.aggregation_response_paper(aggregated_portfolio)

Instrument :LusidInstrumentId=LUID_X6QSC75W/GBP
Units :-98000.0
Current Price :£6.57
Present Value :£-643432.72
Cost :£-577220.0
Return :-11.471%

Instrument :LusidInstrumentId=LUID_JNTQR7TW/GBP
Units :1500000.0
Current Price :£3.55
Present Value :£5327280.0
Cost :£6270000.0
Return :-15.0354%

Instrument :LusidInstrumentId=LUID_XA08PQYW/GBP
Units :265600.0
Current Price :£30.32
Present Value :£8052397.06
Cost :£7686464.0
Return :4.7607%

Instrument :LusidInstrumentId=LUID_418ZP2DT/GBP
Units :345000.0
Current Price :£20.6
Present Value :£7105557.9
Cost :£7590000.0
Return :-6.3826%

Instrument :LusidInstrumentId=LUID_NHW7YM30/GBP
Units :-543000.0
Current Price :£13.82
Present Value :£-7503461.79
Cost :£-7384800.0
Return :-1.6068%

Instrument :LusidInstrumentId=LUID_88ZHDL5K/GBP
Units :98000.0
Current Price :£19.51
Present Value :£1912019.2
Cost :£2156000.0
Return :-11.3164%

Instrument :LusidInstrumentId=LUID_CTAHUMVG/GBP
Units :120000.0
Current Price :£13.36
Present Value :£1603546.8
Co

## 9) Compare performance across the analyst's reference and transaction portfolios

In [16]:
# create index portfolio
index_portfolio_code = 'LSE_FTSE_techMarkFocus_index_DEMO'
prettyprint.heading('Index Portfolio Code', index_portfolio_code)

description = 'Index transaction portfolio_DEMO'
fun.request_transaction_portfolio_creation(index_portfolio_code, 
                                           portfolio_creation_date, 
                                           analyst_scope_code, 
                                           description,
                                           client)


Index Portfolio Code: LSE_FTSE_techMarkFocus_index_DEMO
Portfolio Created
Scope: analyst-paper-37c5-b819-20df-54
Code: LSE_FTSE_techMarkFocus_index_DEMO
Portfolio Effective From: 2019-09-27 13:57:17.996804+00:00
Portfolio Created On: 2019-09-30 13:57:49.511764+00:00



We now have an index portfolio that we can use to hold our index and obtain our benchmark.

![Scopes](img/paper-indexportfolio.gif)

In [17]:
# setup an index portfolio
index_setup = fun.setup_index(analyst_scope_code, reference_portfolio_code, instrument_prices, client)

# Call LUSID to set all of our holdings to the initial index level
adjust_holdings_response = client.transaction_portfolios.set_holdings(
    scope=analyst_scope_code,
    code=index_portfolio_code,
    effective_at=portfolio_creation_date,
    holding_adjustments=index_setup)

prettyprint.set_holdings_response(
    adjust_holdings_response, 
    analyst_scope_code, 
    index_portfolio_code)

Holdings Successfully Set for Portfolio
Scope: analyst-paper-37c5-b819-20df-54
Code: LSE_FTSE_techMarkFocus_index_DEMO
Holdings Effective From: 2019-09-27 13:57:17.996804+00:00
Holdings Created On: 2019-09-30 13:57:55.976065+00:00



We now have an index portfolio that we can value.

![Scopes](img/paper-indexportfoliofilled.gif)

In [18]:
fun.create_analytics_aggregation_request(analyst_scope_code, index_portfolio_code, today, client)


Initial Index Level :999.98
Current Index Level :995.97
Return :-0.4013%
